In [ ]:
from Bio import SeqIO
from glob import glob 
from subprocess import call, check_output

In [ ]:
# enter your params

read_dir = 'orders/current/fall_2016_round_2/genscript_sequencing/191641_dna_seq_rep_9025003_zip/'
wt_pep = 'oligo_design/bglb_model.fa'
# allowed_mutants = 'allowed_mutants.txt' 
sample_list = []

In [ ]:
# convert all the reads to FASTQ

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

In [ ]:
# read from file 

# my_sample_list_file = 'sample_list.txt'
# with open( my_sample_list_file ) as fn:
#     sample_list = fn.read().split()

# try get a ist of the samples if we don't already have that 

# if not sample_list:
#     my_glob = '{}/*fastq'
#     print 'No sample list, attempting to find one with the glob "{}"'.format( my_glob ) 
#     my_files = glob( my_glob.format( read_dir ) )
#     #my_samples = map( lambda x: x.split( '/' )[1].split( '_' )[2], my_files ) 
#     sample_list = [ x.split( '/' )[1].split( '_' )[2] for x in my_files ] 
#     print sample_list 

In [ ]:
# pair and merge the reads 

expect = 0
for sample in sample_list:

    #genscript
    fwd = glob( '{}/*_{}_T7.fastq'.format( read_dir, sample ) ) 
    rev = glob( '{}/*_{}_T7-Ter.fastq'.format( read_dir, sample ) )
    #print( len(fwd), len(rev) )
    
    #operon 
#     fwd = glob( '{}/{}_T7_*.fastq'.format( read_dir, sample ) ) 
#     rev = glob( '{}/{}_T7term_*.fastq'.format( read_dir, sample ) )
#     print len( fwd ), len( rev )

    if len( fwd ) == len( rev ) == 1:
        pear_opts = [ '-e', '2' ] 
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', '{}/{}'.format( read_dir, sample ) ] + pear_opts
        call( cmd )
        expect += 1 

In [ ]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [ ]:
# # load in allowed mutants for error checking 

# with open( allowed_mutants ) as fn:
#     allowed_mutants = [ i.strip() for i in fn.readlines() if len(i) > 1 ] 

In [ ]:
# diff the assemblies against the wild type protein sequence 

singles = []
for assembly in glob( '{}/*.assembled.fasta'.format( read_dir ) ):
    name = assembly.split( '.' )[0] 
    lines = check_output( [ 'blastx', '-subject', wt_pep, '-query', assembly, '-outfmt', '6 sseq qseq length' ] ).split()
    if lines and lines[0] and lines[1] and lines[2]:        
        z = zip( str( lines[0] ), str( lines[1] ) )
        d = [ '{}{}{}'.format( i, n-1, j ) for n, ( i, j ) in enumerate( z ) if i != j ]
        dstr = '+'.join( d )
        # coded responses 
        if len( d ) == 0: 
            print( name, '\twild type' )
        elif len( d ) == 1:
            if d[0] in allowed_mutants:
                print( name, '\t{}'.format( dstr ) )
                singles.append( ( d[0], name ) )
            else:
                print( name, '\t{}'.format( dstr ) )
                #print( name, '\tbad alignment' )
        elif len( d ) > 1:
            #print name, '\t... ({} mutations)'.format( len( d ) ) 
            print( name, '\t{}'.format( dstr ) )

In [ ]:
# results 
picks = dict( singles ) 
l = len( picks ) 
print( 'Found {} of {} expected mutants'.format( l, expect ) )